# Лабораторная работа №3 Разработка ДКА

**Цель:**
Научиться разрабатывать решать задачи с помощью ДКА

**Смысл:**
С помощью ДКА можно .реализовывать конфигурируемые системы

**Задача.**
В зданиие два лифта. На каждом этаже, кроме вернего и первого, есть две кнопки
вызова лифта:

- вызвать лифт для подъема на верхний этаж
- вызвать лифт для спуска на нижний этаж.
На первом соотвественно есть только одна кнопка для вызова лифта для подъема
на верхний этаж, а на последнем, соотвественно, только одна кнопка для вызова
лифта для спуска на нижний этаж. Лифт останавливается на этаже и открывает
двери, только если на этом этаже нажата кнопка вызова, совпадающая с
движением лифта. Каждый лифт обслуживает оба направления движения.


Интерфейс лифта содержит команлы:
- проехать этаж вверх
- открыть двери
- закртыть двери
- проехать этаж вниз

Лифт может двигаться с закртыми дверьми, при поптыке вполнить команду
проехать этаж вверх на посленем этаже и выполнить команду проехать этаж вниз
на первом этаже, команда выбрасывает исключение.
Написать детерминированный конечный автомат для управления движения
лифтами. Стараемся оптимизировать время ожидания пассажиров.

*На вход программа получает* - этажность здания, этажи, на котором располагаются
лифты, список пар (номер этажа вызова, номер этажа. кужа нужно попасть).

*На выходе* - последовательность команд каждого из лифтов, для каждого вызова
необходимо вывести количество перемещений между этажами, которые совершили
лифты до отытия дверей на этаже вызова.

In [28]:
floors = 20

class Elevator:
    def __init__(self, id, cur_floor):
        self.id = id
        self.current_floor = cur_floor
        self.state = "waiting"
        self.num_moves = 0
        self.call_queue = []

    def open_doors(self):
        self.state = "waiting"

    def passenger_entered(self, floor):
        self.call_queue.append(floor)

    def passenger_got_out(self):
        self.call_queue.pop(0)

    def move_up(self):
        self.current_floor += 1
        self.num_moves += 1
        if self.current_floor == floors:
            self.state = "waiting"
        else:
            self.state = "movingUp"

    def move_down(self):
        self.current_floor -= 1
        self.num_moves += 1
        if self.current_floor == 1:
            self.state = "waiting"
        else:
            self.state = "movingDown"

    def process_call(self, floor):
        if self.state == "waiting":
            if floor > self.current_floor:
                self.move_up()
            elif floor < self.current_floor:
                self.move_down()
            else:
                self.open_doors()
        elif self.state == "movingUp":
            if floor > self.current_floor:
                self.move_up()
            elif floor == self.current_floor:
                self.open_doors()
            else:
                self.move_down()
        elif self.state == "movingDown":
            if floor < self.current_floor:
                self.move_down()
            elif floor == self.current_floor:
                self.open_doors()
            else:
                self.move_up()
    


In [36]:
building = [
    Elevator(1, 1),
    Elevator(2, floors),
]

call_queue = [(4, 6), (5, 3), (10, 1), (6, 3), (9, 1), (5, 4), (2, 9), (20, 1), (15, 20), (12, 4), (13, 6), (14, 17)]
call_queue_c = call_queue.copy()

for call in call_queue_c:
    if call not in call_queue:
        continue
    min_distance = float("inf")
    closest_elevator = None
    if call[1] - call[0] == 0 or call[0] < 0 or call[1] < 0 or call[0] > floors or call[1] > floors:
        call_queue.remove(call)

    for elevator in building:
        distance = abs(elevator.current_floor - call[0])
        if distance < min_distance:
            min_distance = distance
            closest_elevator = elevator

    while closest_elevator.current_floor != call[0]:
        closest_elevator.process_call(call[0])
        call_queue_copy = call_queue.copy()
        for other_call in call_queue_copy:
            if other_call == call:
                continue
            diff = other_call[1] - other_call[0]
            direction = "movingUp" if diff > 0 else "movingDown"
            if other_call[0] == closest_elevator.current_floor and direction == closest_elevator.state:
                closest_elevator.passenger_entered(other_call[1])
                call_queue.remove(other_call)
            if closest_elevator.current_floor in closest_elevator.call_queue:
                closest_elevator.call_queue.remove(closest_elevator.current_floor)

    closest_elevator.passenger_entered(call[1])
    if call in call_queue:
        call_queue.remove(call)

    while len(closest_elevator.call_queue) != 0:
        passenger_floor = closest_elevator.call_queue.pop(0)

        while passenger_floor != closest_elevator.current_floor:
            closest_elevator.process_call(passenger_floor)
            call_queue_copy = call_queue.copy()
            for other_call in call_queue_copy:
                if other_call == call:
                    continue
                diff = other_call[1] - other_call[0]
                direction = None
                if diff > 0:
                    direction = "movingUp"
                else:
                    direction = "movingDown"
                if other_call[0] == closest_elevator.current_floor and direction == closest_elevator.state:
                    closest_elevator.passenger_entered(other_call[1])
                    call_queue.remove(other_call)

            if closest_elevator.current_floor in closest_elevator.call_queue:
                closest_elevator.call_queue.remove(closest_elevator.current_floor)
                print(
                    f"Лифт #{closest_elevator.id} забрал пассажира по пути"
                    f" и доставил на этаж {closest_elevator.current_floor}, количество действий {closest_elevator.num_moves}")

    print(
        f"Лифт #{closest_elevator.id} забрал пассажира с этажа {call[0]}"
        f" и доставил на этаж {call[1]}, количество действий {closest_elevator.num_moves}, "
        f"сейчас лифт на этаже {closest_elevator.current_floor}")


Лифт #1 забрал пассажира с этажа 4 и доставил на этаж 6, количество действий 5, сейчас лифт на этаже 6
Лифт #1 забрал пассажира с этажа 5 и доставил на этаж 3, количество действий 8, сейчас лифт на этаже 3
Лифт #2 забрал пассажира с этажа 10 и доставил на этаж 1, количество действий 11, сейчас лифт на этаже 1
Лифт #1 забрал пассажира с этажа 6 и доставил на этаж 3, количество действий 14, сейчас лифт на этаже 3
Лифт #1 забрал пассажира с этажа 9 и доставил на этаж 1, количество действий 28, сейчас лифт на этаже 1
Лифт #1 забрал пассажира с этажа 5 и доставил на этаж 4, количество действий 33, сейчас лифт на этаже 4
Лифт #2 забрал пассажира с этажа 2 и доставил на этаж 9, количество действий 19, сейчас лифт на этаже 9
Лифт #2 забрал пассажира с этажа 20 и доставил на этаж 1, количество действий 49, сейчас лифт на этаже 1
Лифт #1 забрал пассажира с этажа 15 и доставил на этаж 20, количество действий 49, сейчас лифт на этаже 20
Лифт #1 забрал пассажира с этажа 12 и доставил на этаж 4, кол